In [1]:
from ENDFtk.tree import Tape
from NDSampler import NDSampler, SamplerSettings, generate_covariance_dict
import numpy as np
import glob
import matplotlib.pyplot as plt

endf_tape = Tape.from_file('/home/sole-pie01/ndlib/endfb8-neutron/n-010_Ne_021.endf')
covariance_dict = generate_covariance_dict(endf_tape)
# del covariance_dict[31]
del covariance_dict[33]
del covariance_dict[34]
del covariance_dict[32][151][0] #No RRR
# del covariance_dict[35]
covariance_dict

{32: {151: [1]}}

In [3]:
samplerSettings = SamplerSettings(sampling='LHS', widths_to_reduced=True, debug=True)
# covariance_data_Pu9.hdf5
sampler = NDSampler(endf_tape, covariance_dict=covariance_dict, settings=samplerSettings) #, hdf5_filename="covariance_data_Pu9.hdf5")
sampler.sample(num_samples = 500)  # This will create files 'sampled_tape_1.endf', 'sampled_tape_2.endf', etc.

Processing MF=32, MT=151 with NER list: [1]
Processing 1 resonance range(s) with NER values: [1]
Processing NER=1 with LRU=2, LRF=2
Creating Unresolved Breit-Wigner covariance object for NER=1
Time for extracting covariance matrix: 0.0011 seconds
Time for compute_L_matrix: 0.0023 seconds
Created 1 resonance covariance objects
Generating 500 samples using LHS method...

=== Debug Output for URR Breit-Wigner (Transformed Samples) ===
Number of parameters: 36
Number of samples: 500
Sampling method: LHS

Transformed sample matrix (first 5 samples, first 10 parameters):
Sample 1: [4.84920545e+04 1.52007175e+01 4.76354937e+04 1.58116011e+01
 4.65709608e+04 1.28033847e+01 4.08402227e+04 1.28115802e+01
 4.03892427e+04 1.29235612e+01]
Sample 2: [4.80805668e+04 1.52168328e+01 4.80205230e+04 1.45440506e+01
 4.78416105e+04 1.51787966e+01 4.05076737e+04 9.96875235e+00
 4.04907134e+04 1.09310003e+01]
Sample 3: [4.87322350e+04 1.35118121e+01 4.75713659e+04 1.60063731e+01
 4.77260306e+04 1.82046547e+0

In [47]:
MAT = endf_tape.MAT(endf_tape.material_numbers[0])
mf32 = endf_tape.MAT(endf_tape.material_numbers[0]).MF(32).MT(151).parse()
mf32_resonance_range = mf32.isotopes[0].resonance_ranges[1]

NPAR_spin = mf32_resonance_range.parameters.covariance_matrix.NPAR  # Number of parameters at the spin group level
# Get the spin-level covariance matrix
relative_cov_matrix_spin = np.zeros((NPAR_spin, NPAR_spin))

# Use numpy indexing for direct assignment to upper triangle
triu_indices = np.triu_indices(NPAR_spin)
relative_cov_matrix_spin[triu_indices] = mf32_resonance_range.parameters.covariance_matrix.covariance_matrix

# Make it symmetric
relative_cov_matrix_spin = relative_cov_matrix_spin + relative_cov_matrix_spin.T - np.diag(np.diag(relative_cov_matrix_spin))
np.set_printoptions(precision=4, suppress=False, linewidth=100)
print(relative_cov_matrix_spin)

[[0.0001 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.01   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.0001 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.01   0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.0001 0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.01   0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.0001 0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.01   0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.0001 0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.01   0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.0001 0.    ]
 [0.     0.     0.     0.     0.     0.    

In [37]:
def reconstruct_covariance_multiLJ(self, LJ_structure, relative_cov_matrices, nominal_values, MPAR=3):
    """
    Reconstruct full covariance matrix for multiple (L,J) groups.
    
    Parameters
    ----------
    LJ_structure : dict
        Nested dict {L: {J: NEnergies}}, for example:
        {0: {0: 2, 1: 3}, 1: {0: 1}} means:
        - L=0: J=0 has 2 energies, J=1 has 3 energies
        - L=1: J=0 has 1 energy
    
    relative_cov_matrices : dict
        Dict {(L,J): rel_cov_matrix} with rel_cov_matrix shape (MPAR, MPAR)
    
    nominal_values : ndarray
        All parameters sequentially stored by increasing energies, then J, then L.
    
    MPAR : int
        Number of parameters per energy (default = 3).
    
    Returns
    -------
    full_cov_matrix : ndarray
        Absolute covariance matrix of shape (N_total_params, N_total_params).
    """
    
    total_params = nominal_values.size
    full_cov_matrix = np.zeros((total_params, total_params))

    current_idx = 0

    for L in sorted(LJ_structure.keys()):
        for J in sorted(LJ_structure[L].keys()):
            NE = LJ_structure[L][J]
            block_size = MPAR * NE
            
            # Extract nominal values block for current (L,J) group
            nominal_block = nominal_values[current_idx: current_idx + block_size]
            
            # Get corresponding relative covariance matrix
            rel_cov = relative_cov_matrices[(L,J)]

            # Compute covariance block for current (L,J)
            cov_block = np.zeros((block_size, block_size))

            for ek in range(NE):
                for ekp in range(NE):
                    idx_k = slice(MPAR*ek, MPAR*(ek+1))
                    idx_kp = slice(MPAR*ekp, MPAR*(ekp+1))

                    # Outer product for current energy pair
                    nominal_outer = np.outer(nominal_block[idx_k], nominal_block[idx_kp])

                    # Absolute covariance from relative covariance
                    cov_block[idx_k, idx_kp] = rel_cov * nominal_outer
            
            # Insert block into the global covariance matrix
            idx_slice = slice(current_idx, current_idx + block_size)
            full_cov_matrix[idx_slice, idx_slice] = cov_block
            
            # Update index for next (L,J) group
            current_idx += block_size
    
    return full_cov_matrix

MAT = endf_tape.MAT(endf_tape.material_numbers[0])
mf2 = endf_tape.MAT(endf_tape.material_numbers[0]).MF(2).MT(151).parse()
mf2_range = mf2.isotopes[0].resonance_ranges[1]

l_j_ne_dict = {}
for l_index, l_value in enumerate(mf2_range.parameters.l_values.to_list()):
    l_j_ne_dict[l_index] = {}
    for j_index, j_value in enumerate(l_value.j_values.to_list()):
        l_j_ne_dict[l_index][j_index] = j_value.NE
print(l_j_ne_dict)
nominal_values = np.array([
    4.818525e+04, 1.480170e+01, 4.777825e+04, 1.459500e+01, 4.737450e+04,
    1.439310e+01, 4.073475e+04, 1.251310e+01, 4.039075e+04, 1.233830e+01,
    4.004950e+04, 1.216770e+01, 1.217822e+05, 1.129530e+02, 1.207535e+05,
    1.122750e+02, 1.197338e+05, 1.115620e+02, 4.818525e+04, 3.582630e+01,
    4.777825e+04, 3.563540e+01, 4.737450e+04, 3.543470e+01, 4.073475e+04,
    3.028690e+01, 4.039075e+04, 3.012550e+01, 4.004950e+04, 2.995580e+01,
    4.866200e+04, 4.513370e+01, 4.825075e+04, 4.486300e+01, 4.784325e+04,
    4.457810e+01])

comat = reconstruct_covariance_multiLJ(sampler, l_j_ne_dict, relative_cov_matrix_spin, nominal_values, MPAR=2)
with np.printoptions(precision=2, suppress=True, linewidth=100): 
    print(comat)

{0: {0: 3, 1: 3}, 1: {0: 3, 1: 3, 2: 3, 3: 3}}
[[232181.83     71.32 230220.69 ...      0.        0.        0.  ]
 [    71.32      0.02     70.72 ...      0.        0.        0.  ]
 [230220.69     70.72 228276.12 ...      0.        0.        0.  ]
 ...
 [     0.        0.        0.   ...      0.        0.        0.  ]
 [     0.        0.        0.   ...      0.        0.        0.  ]
 [     0.        0.        0.   ...      0.        0.        0.  ]]


In [35]:
def reconstruct_covariance_multiLJ(LJ_structure, relative_cov_matrices, nominal_values, MPAR=3, fully_correlate_energies=False):
    """
    Reconstruct full covariance matrix for multiple (L,J) groups with no correlations between energies.
    
    Parameters
    ----------
    LJ_structure : dict
        Nested dict {L: {J: NEnergies}}, e.g. {0: {0:2}}.
    
    relative_cov_matrices : dict
        {(L,J): rel_cov_matrix} with shape (MPAR, MPAR).
    
    nominal_values : ndarray
        Parameter values ordered as [p1(E0), p2(E0),..., p1(E1), p2(E1),..., per (L,J)].
    
    MPAR : int
        Number of parameters per energy.
    
    Returns
    -------
    full_cov_matrix : ndarray
        Absolute covariance matrix, block diagonal per energy.
    """
    if fully_correlate_energies:
        return reconstruct_covariance_multiLJ_correlated_E(LJ_structure, relative_cov_matrices, nominal_values, MPAR)
    
    total_params = nominal_values.size
    full_cov_matrix = np.zeros((total_params, total_params))

    idx_global = 0  # Tracks global index position

    for L in sorted(LJ_structure.keys()):
        for J in sorted(LJ_structure[L].keys()):
            NE = LJ_structure[L][J]
            rel_cov = relative_cov_matrices[(L,J)]

            for ek in range(NE):
                idx = slice(idx_global, idx_global + MPAR)
                p_nominal = nominal_values[idx]

                # Compute absolute covariance block for energy ek
                nominal_outer = np.outer(p_nominal, p_nominal)
                cov_block = rel_cov * nominal_outer

                # Place the covariance block on the diagonal (no cross-energy correlations)
                full_cov_matrix[idx, idx] = cov_block

                idx_global += MPAR

    return full_cov_matrix


def reconstruct_covariance_multiLJ_correlated_E(LJ_structure, relative_cov_matrices, nominal_values, MPAR=3):
    """
    Reconstruct full covariance matrix for multiple (L,J) groups.
    
    Parameters
    ----------
    LJ_structure : dict
        Nested dict {L: {J: NEnergies}}, for example:
        {0: {0: 2, 1: 3}, 1: {0: 1}} means:
        - L=0: J=0 has 2 energies, J=1 has 3 energies
        - L=1: J=0 has 1 energy
    
    relative_cov_matrices : dict
        Dict {(L,J): rel_cov_matrix} with rel_cov_matrix shape (MPAR, MPAR)
    
    nominal_values : ndarray
        All parameters sequentially stored by increasing energies, then J, then L.
    
    MPAR : int
        Number of parameters per energy (default = 3).
    
    Returns
    -------
    full_cov_matrix : ndarray
        Absolute covariance matrix of shape (N_total_params, N_total_params).
    """
    
    total_params = nominal_values.size
    full_cov_matrix = np.zeros((total_params, total_params))

    current_idx = 0

    for L in sorted(LJ_structure.keys()):
        for J in sorted(LJ_structure[L].keys()):
            NE = LJ_structure[L][J]
            block_size = MPAR * NE
            
            # Extract nominal values block for current (L,J) group
            nominal_block = nominal_values[current_idx: current_idx + block_size]
            
            # Get corresponding relative covariance matrix
            rel_cov = relative_cov_matrices[(L,J)]

            # Compute covariance block for current (L,J)
            cov_block = np.zeros((block_size, block_size))

            for ek in range(NE):
                for ekp in range(NE):
                    idx_k = slice(MPAR*ek, MPAR*(ek+1))
                    idx_kp = slice(MPAR*ekp, MPAR*(ekp+1))

                    # Outer product for current energy pair
                    nominal_outer = np.outer(nominal_block[idx_k], nominal_block[idx_kp])

                    # Absolute covariance from relative covariance
                    cov_block[idx_k, idx_kp] = rel_cov * nominal_outer
            
            # Insert block into the global covariance matrix
            idx_slice = slice(current_idx, current_idx + block_size)
            full_cov_matrix[idx_slice, idx_slice] = cov_block
            
            # Update index for next (L,J) group
            current_idx += block_size
    
    return full_cov_matrix

l_j_ne_dict = {0: {0: 2}}
MPAR = 2
nominal_values = np.array([1.0, 2.0, 3.0, 4.0]) # in order [D(E=0, L=0, J=0), GN(E=0, L=0, J=0), D(E=1, L=0, J=0), GN(E=1, L=0, J=0)]
relative_matrix = np.array([[0.1, 0], # D variance for (E=:, L=0, J=0)
                            [0, 0.2]]) # GN variance for (E=:, L=0, J=0)

# In this case MPAR=2 indicates that the true matrix paraemeters are organized in this order [D, GN] for each (E,L,J)
# If MPAR=3, [D, GN, GG], MPAR=4, [D, GN, GG, GF], etc.
# In our case reconstruct the true matrix [D(E=0, L=0, J=0), GN(E=0, L=0, J=0), D(E=1, L=0, J=0), GN(E=1, L=0, J=0)]
covariance_matrix = reconstruct_covariance_multiLJ(l_j_ne_dict, relative_matrix, nominal_values, MPAR, fully_correlate_energies=False)
print(covariance_matrix)

# Std deviations
DD0  = relative_matrix[0][0]*nominal_values[0]
DGN0 = relative_matrix[1][1]*nominal_values[1]
DD1  = relative_matrix[0][0]*nominal_values[2]
DGN1 = relative_matrix[1][1]*nominal_values[3]

true_matrix = np.array([[DD0, 0, 0, 0],
                        [0, DGN0, 0, 0],
                        [0, 0, DD1, 0],
                        [0, 0, 0, DGN1]])
print(true_matrix)

[[0.1 0.2 0.  0. ]
 [0.2 0.4 0.  0. ]
 [0.  0.  0.9 1.2]
 [0.  0.  1.2 1.6]]
[[0.1 0.  0.  0. ]
 [0.  0.4 0.  0. ]
 [0.  0.  0.3 0. ]
 [0.  0.  0.  0.8]]


In [36]:
# Simplified example given by you:
l_j_ne_dict = {0: {0: 2}}
MPAR = 2
nominal_values = np.array([1.0, 2.0, 3.0, 4.0])  # [D0,GN0,D1,GN1]

relative_matrix = np.array([[0.1, 0],
                            [0, 0.2]])

relative_cov_matrices = {(0,0): relative_matrix}

# Corrected covariance reconstruction
covariance_matrix = reconstruct_covariance_multiLJ(l_j_ne_dict, relative_cov_matrices, nominal_values, MPAR)
print("Covariance matrix (corrected):\n", covariance_matrix)

# Expected true matrix (as you defined explicitly):
true_matrix = np.array([[0.1*(1.0)**2, 0, 0, 0],
                        [0, 0.2*(2.0)**2, 0, 0],
                        [0, 0, 0.1*(3.0)**2, 0],
                        [0, 0, 0, 0.2*(4.0)**2]])
print("\nTrue covariance matrix:\n", true_matrix)


Covariance matrix (corrected):
 [[0.1 0.  0.  0. ]
 [0.  0.8 0.  0. ]
 [0.  0.  0.9 0. ]
 [0.  0.  0.  3.2]]

True covariance matrix:
 [[0.1 0.  0.  0. ]
 [0.  0.8 0.  0. ]
 [0.  0.  0.9 0. ]
 [0.  0.  0.  3.2]]


In [55]:
def reconstruct_covariance_multiLJ(LJ_structure, relative_cov_matrix, nominal_values, MPAR=2):
    """
    Construct full covariance matrix given energy-independent relative covariance.

    Parameters:
    -----------
    LJ_structure: dict
        Structure {L: {J: NEnergies}}.

    relative_cov_matrix: ndarray
        Relative covariance matrix (MPAR*N(L,J), MPAR*N(L,J)).

    nominal_values: ndarray
        Nominal parameters ordered by [E increasing, J increasing, L increasing].

    MPAR: int
        Number of parameters per energy.

    Returns:
    --------
    full_cov_matrix: ndarray
        Absolute covariance matrix (Nparams, Nparams).
    """

    total_params = nominal_values.size
    full_cov_matrix = np.zeros((total_params, total_params))

    # Build a list of indices per (L,J) to map correctly
    LJ_indices = []
    idx = 0
    for L in sorted(LJ_structure.keys()):
        for J in sorted(LJ_structure[L].keys()):
            NE = LJ_structure[L][J]
            indices = [idx + MPAR*e for e in range(NE)]
            LJ_indices.append((L,J,indices))
            idx += MPAR * NE

    N_LJ = len(LJ_indices)

    # Loop to construct blocks
    for lj1_idx, (L1,J1,E_indices1) in enumerate(LJ_indices):
        for lj2_idx, (L2,J2,E_indices2) in enumerate(LJ_indices):
            # Relative covariance block
            rel_block = relative_cov_matrix[MPAR*lj1_idx:MPAR*(lj1_idx+1),
                                            MPAR*lj2_idx:MPAR*(lj2_idx+1)]

            for ei1, idx1 in enumerate(E_indices1):
                for ei2, idx2 in enumerate(E_indices2):
                    abs_block = rel_block * np.outer(
                        nominal_values[idx1:idx1+MPAR],
                        nominal_values[idx2:idx2+MPAR]
                    )

                    # If energies differ, set covariance zero
                    if (L1,J1,ei1) != (L2,J2,ei2):
                        abs_block *= 0

                    # Place block
                    full_cov_matrix[idx1:idx1+MPAR, idx2:idx2+MPAR] = abs_block

    return full_cov_matrix

# Your LJ structure
l_j_ne_dict = {0: {0: 3, 1: 3}, 1: {0: 3, 1: 3, 2: 3, 3: 3}}

# MPAR=2 parameters per energy
MPAR = 2

# Your nominal values (must match the exact ordering [E increasing, J increasing, L increasing])
# For instance, nominal_values.shape = (36,) as (total energies=18)*(MPAR=2)
nominal_values = np.array([
    # L=0, J=0, E=0,1,2
    1,2, 3,4, 5,6,
    # L=0, J=1, E=0,1,2
    7,8, 9,10, 11,12,
    # L=1, J=0, E=0,1,2
    13,14, 15,16, 17,18,
    # L=1, J=1, E=0,1,2
    19,20, 21,22, 23,24,
    # L=1, J=2, E=0,1,2
    25,26, 27,28, 29,30,
    # L=1, J=3, E=0,1,2
    31,32, 33,34, 35,36
])

# Your relative covariance matrix (12x12)
relative_matrix = relative_cov_matrix_spin

# Call the reconstruction function
covariance_matrix = reconstruct_covariance_multiLJ(
    l_j_ne_dict, relative_matrix, nominal_values, MPAR
)
import pandas as pd
covariance_matrix = pd.DataFrame(covariance_matrix)
pd.set_option('display.float_format', '{:.4f}'.format, 'display.width', 1000)
print(covariance_matrix)


       0      1      2      3      4      5      6      7      8      9      10     11     12     13     14     15     16     17     18     19     20     21     22     23     24     25     26     27     28     29     30      31     32      33     34      35
0  0.0001 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000  0.0000 0.0000  0.0000
1  0.0000 0.0400 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000  0.0000 0.0000  0.0000
2  0.0000 0.0000 0.0009 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000  0.000

In [ ]:
import numpy as np

def expand_relative_covariance(LJ_structure, relative_cov_matrix, MPAR=2, fully_correlated_energies=True):
    """
    Expand relative covariance matrix from (L,J,param) to (L,J,energy,param).

    Parameters:
    -----------
    LJ_structure: dict
        {L: {J: NEnergies}}.

    relative_cov_matrix: ndarray
        Relative covariance matrix shape (NPAR, NPAR), with NPAR=MPAR*N(L,J).

    MPAR: int
        Number of parameters per energy.

    fully_correlated_energies: bool
        If True, parameters fully correlated across energies.
        If False, parameters correlated only within the same energy.

    Returns:
    --------
    expanded_rel_cov_matrix: ndarray
        Expanded covariance matrix shape (N_total_params, N_total_params).
    """

    LJ_list = [(L, J) for L in sorted(LJ_structure) for J in sorted(LJ_structure[L])]
    NJS = len(LJ_list)
    expanded_dim = MPAR * sum(LJ_structure[L][J] for L, J in LJ_list)

    expanded_rel_cov_matrix = np.zeros((expanded_dim, expanded_dim))

    orig_i = 0
    exp_i = 0
    for L1, J1 in LJ_list:
        NE1 = LJ_structure[L1][J1]
        for e1 in range(NE1):
            orig_j = 0
            exp_j = 0
            for L2, J2 in LJ_list:
                NE2 = LJ_structure[L2][J2]
                cov_block = relative_cov_matrix[
                    orig_i:orig_i + MPAR,
                    orig_j:orig_j + MPAR
                ]

                for e2 in range(NE2):
                    if fully_correlated_energies or ((L1, J1, e1) == (L2, J2, e2)):
                        expanded_rel_cov_matrix[
                            exp_i:exp_i + MPAR,
                            exp_j:exp_j + MPAR
                        ] = cov_block
                    else:
                        expanded_rel_cov_matrix[
                            exp_i:exp_i + MPAR,
                            exp_j:exp_j + MPAR
                        ] = np.zeros((MPAR, MPAR))
                    exp_j += MPAR
                orig_j += MPAR
            exp_i += MPAR
        orig_i += MPAR

    return expanded_rel_cov_matrix


print(pd.DataFrame(expand_relative_covariance(l_j_ne_dict, relative_cov_matrix_spin, MPAR=2)))

       0      1      2      3      4      5      6      7      8      9      10     11     12     13     14     15     16     17     18     19     20     21     22     23     24     25     26     27     28     29     30     31     32     33     34     35
0  0.0001 0.0000 0.0001 0.0000 0.0001 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000
1  0.0000 0.0100 0.0000 0.0100 0.0000 0.0100 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000
2  0.0001 0.0000 0.0001 0.0000 0.0001 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0

In [ ]:
l_j_ne_dict = {}
for l_index, l_value in enumerate(mf2_range.parameters.l_values.to_list()):
    l_j_ne_dict[l_index] = {}
    for j_index, j_value in enumerate(l_value.j_values.to_list()):
        l_j_ne_dict[l_index][j_index] = j_value.NE
print(l_j_ne_dict)
nominal_values = np.array([
    4.818525e+04, 1.480170e+01, 4.777825e+04, 1.459500e+01, 4.737450e+04,
    1.439310e+01, 4.073475e+04, 1.251310e+01, 4.039075e+04, 1.233830e+01,
    4.004950e+04, 1.216770e+01, 1.217822e+05, 1.129530e+02, 1.207535e+05,
    1.122750e+02, 1.197338e+05, 1.115620e+02, 4.818525e+04, 3.582630e+01,
    4.777825e+04, 3.563540e+01, 4.737450e+04, 3.543470e+01, 4.073475e+04,
    3.028690e+01, 4.039075e+04, 3.012550e+01, 4.004950e+04, 2.995580e+01,
    4.866200e+04, 4.513370e+01, 4.825075e+04, 4.486300e+01, 4.784325e+04,
    4.457810e+01])

comat = reconstruct_covariance_multiLJ(sampler, l_j_ne_dict, relative_cov_matrix_spin, nominal_values, MPAR=2)
with np.printoptions(precision=2, suppress=True, linewidth=100): 
    print(comat)

In [ ]:
import pandas as pd

transformed_df = pd.read_csv('transformed_samples_URR_BW.csv')
transformed_df.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# The dataset is already loaded as transformed_df
# First row contains nominal values, second row contains uncertainties, rows 3+ contain samples

# Extract the first row (nominal values) and second row (uncertainties)
nominal_values = transformed_df.iloc[0].drop('# Row')
uncertainties = transformed_df.iloc[1].drop('# Row')
sample_means = transformed_df.loc[2].drop('# Row')
sample_stds = transformed_df.loc[3].drop('# Row')
meanpctdiff = transformed_df.iloc[4].drop('# Row')
stdpctdiff = transformed_df.iloc[5].drop('# Row')

# Calculate statistics on the samples (rows 6 and beyond)
actual_samples = transformed_df.iloc[6:].set_index('# Row')

# Create a comparison DataFrame
comparison_df = pd.DataFrame({
    'Nominal': nominal_values,
    'Sample Mean': sample_means,
    'Mean % Diff': meanpctdiff,
    'Expected STD': uncertainties,
    'Sample STD': sample_stds,
    'STD % Diff': stdpctdiff
})

# Sort by absolute percentage difference in means to highlight the most "off" parameters
comparison_df['Abs Mean % Diff'] = abs(comparison_df['Mean % Diff'])
comparison_df['Abs STD % Diff'] = abs(comparison_df['STD % Diff'])
comparison_df_sorted = comparison_df #.sort_values('Abs Mean % Diff', ascending=False)

# Print the sorted comparison to highlight the parameters with largest deviations
print("Comparison of statistics (sorted by absolute mean % difference):")
pd.set_option('display.float_format', '{:.3f}'.format, 'display.width', 1000)
print(comparison_df_sorted[['Nominal', 'Sample Mean', 'Mean % Diff', 'Expected STD', 'Sample STD', 'STD % Diff']])

# Calculate the average relative difference for means and standard deviations
mean_rel_diff = abs((nominal_values - sample_means) / nominal_values).mean() * 100
std_rel_diff = abs((uncertainties - sample_stds) / uncertainties).mean() * 100

print(f"\nAverage relative difference in means: {mean_rel_diff:.2f}%")
print(f"Average relative difference in standard deviations: {std_rel_diff:.2f}%")

In [ ]:
# Compute the correlation matrix using the samples
corr_matrix = actual_samples.corr()

# Display the correlation matrix as a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Sampled Parameters')
plt.tight_layout()
plt.show()
